In [1]:
using Distributed
@everywhere using StaticArrays,BenchmarkTools,Random,Flux,DelimitedFiles,LinearAlgebra
using InvertedIndices
@everywhere using MachineLearningPotential

In [2]:
Threads.nthreads()

8

# Symmetry Functions

Time to write and test the basics of SymmFunctions for ML potentials

In [3]:
atoms = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]*18.8973*0.36258

positions = [SVector{3}(p[i] for i in 1:3) for p in atoms]
dis2mat = get_distance2_mat(positions)
rad_function = RadialType2{Float64}(0.001,11.338,[1.,1.])
ang_func = AngularType3{Float64}(0.0001,1.0,1.0,11.338,[1.,1.,1.])
ang_func2 = AngularType3{Float64}(0.0001,-1.0,2.0,11.338,[1.,1.,1.])

AngularType3{Float64}(0.0001, -1.0, 2.0, 11.338, [1.0, 1.0, 1.0])

Demonstrates the difference between [:,index] and [index,:]

# Angular Symmetry Function

Testing prior to type system

In [4]:
@benchmark calc_symmetry_function($positions,$dis2mat,rand(1:55),rad_function)

BenchmarkTools.Trial: 10000 samples with 201 evaluations.
 Range (min … max):  387.761 ns …  17.235 μs  ┊ GC (min … max): 0.00% … 95.87%
 Time  (median):     465.709 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   557.351 ns ± 805.215 ns  ┊ GC (mean ± σ):  7.08% ±  4.78%

    ▁▅█▆                                                         
  ▁▄█████▆▃▂▂▁▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▂▂▃▄▅▇▇▇▆▅▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁ ▂
  388 ns           Histogram: frequency by time          776 ns <

 Memory estimate: 1.06 KiB, allocs estimate: 4.

In [5]:
@benchmark calc_symmetry_function($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  21.271 μs …  1.350 ms  ┊ GC (min … max): 0.00% … 96.43%
 Time  (median):     27.773 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   29.563 μs ± 18.936 μs  ┊ GC (mean ± σ):  0.87% ±  1.37%

               ▅█▇▅         ▂▃▂                                
  ▁▃▅▇▇▅▃▁▁▁▂▄█████▇▄▃▂▂▂▃▅█████▅▄▃▂▁▂▂▂▃▄▄▄▃▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁ ▃
  21.3 μs         Histogram: frequency by time        43.3 μs <

 Memory estimate: 23.39 KiB, allocs estimate: 4.

In [6]:
calc_symmetry_function(positions,dis2mat,1,ang_func)

35.926201081721075

In [7]:
using Distributed
Base.Threads.nthreads()

8

In [8]:
@everywhere import MachineLearningPotential.SymmetryFunctions.calc_one_symm_func


# Benchmarking ASF

ASF was ~35 ${\mu}$ s, now down to ~26

In [9]:
# @everywhere function calc_thr_symm_func(positions,dis2_mat,index,symmfunc::AngularType3)
#     g_vec = []
#     #g_vec = [calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) for j=(1:55) if j !=index for k= 1:j-1 if k != index]
#     for j=(1:55)
#         if j != index
#             for k  = (1:j-1)# if k!=index)
#                 if k!= index
#                     push!(g_vec, calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) )
#                 end            
#             end
#         end
#     end

    
#     return sum(g_vec)*2^(1-symmfunc.zeta)
# end
# @benchmark calc_thr_symm_func($positions,$dis2mat,rand(1:55),ang_func)

In [10]:
# calc_thr_symm_func(positions,dis2mat,1,ang_func)
# @everywhere function calc_thr_symm_func2(positions,dis2_mat,index,symmfunc::AngularType3)
#     g_vec = zeros(1485)
#     for j=(1:55)
#         if j != index
#             for k  = (1:j-1)# if k!=index)
#                 if k!= index
#                     ind = Int((j^2 - 3j)/2 + 1 + k)
#                     g_vec[ind] = calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) 
#                 end            
#             end
#         end
#     end

    
#     return sum(g_vec)*2^(1-symmfunc.zeta)
# end
# calc_thr_symm_func2(positions,dis2mat,1,ang_func)
# @benchmark calc_thr_symm_func2($positions,$dis2mat,rand(1:55),ang_func)

In [11]:
@everywhere function calc_thr_symm_func(positions,dis2_mat,index,symmfunc::AngularType3)
    g_vec = zeros(1485)
    Threads.@threads for j=(1:55)
        if j != index
            for k  = (1:j-1)# if k!=index)
                if k!= index
                    ind = Int((j^2 - 3j)/2 + 1 + k)
                    g_vec[ind] = calc_one_symm_func(positions[index],positions[j],positions[k],dis2_mat[index,j],dis2_mat[index,k],dis2_mat[j,k],symmfunc) 
                end            
            end
        end
    end

    
    return sum(g_vec)*2^(1-symmfunc.zeta)
end

In [12]:
calc_thr_symm_func(positions,dis2mat,1,ang_func)

35.926201081721075

In [13]:
@benchmark calc_thr_symm_func($positions,$dis2mat,rand(1:55),ang_func)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   8.377 μs …  9.022 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     12.469 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.890 μs ± 95.314 μs  ┊ GC (mean ± σ):  1.60% ± 0.98%

  ▂▆███▇▆▅▄▃▃▂▁▁                      ▁▁▁ ▁                   ▂
  ██████████████▇██▇▇▇██▇▇▇▇▇██▇▇███████████████████▇█▇▇▇█▆▆▆ █
  8.38 μs      Histogram: log(frequency) by time      61.7 μs <

 Memory estimate: 16.47 KiB, allocs estimate: 50.

# calculating all the symmetry functions

In [14]:
X = [ 1    1              0.001   0.000  11.338
 1    0              0.001   0.000  11.338
 1    1              0.020   0.000  11.338
 1    0              0.020   0.000  11.338
 1    1              0.035   0.000  11.338
 1    0              0.035   0.000  11.338
 1    1              0.100   0.000  11.338
 1    0              0.100   0.000  11.338
 1    1              0.400   0.000  11.338
 1    0              0.400   0.000  11.338]

radsymmvec = []

for row in eachrow(X)
    symmfunc = RadialType2{Float64}(row[3],row[5],[row[1],row[2]])
    push!(radsymmvec,symmfunc)
end

V = [[0.0001,1,1,11.338],[0.0001,-1,2,11.338],[0.003,-1,1,11.338],[0.003,-1,2,11.338],[0.008,-1,1,11.338],[0.008,-1,2,11.228],[0.008,1,2,11.338],[0.015,1,1,11.338],[0.015,-1,2,11.338],[0.015,-1,4,11.338],[0.015,-1,16,11.338],[0.025,-1,1,11.338],[0.025,1,1,11.338],[0.025,1,2,11.338],[0.025,-1,4,11.338],[0.025,-1,16,11.338],[0.025,1,16,11.338],[0.045,1,1,11.338],[0.045,-1,2,11.338],[0.045,-1,4,11.338],[0.045,1,4,11.338],[0.045,1,16,11.338],[0.08,1,1,11.338],[0.08,-1,2,11.338],[0.08,-1,4,11.338],[0.08,1,4,11.338]]

T = [[1.,1.,1.],[1.,1.,0.],[1.,0.,0.]]

angularsymmvec = []

for element in V 
    for types in T
        symmfunc = AngularType3{Float64}(element[1],element[2],element[3],11.338,types)
        push!(angularsymmvec,symmfunc)
    end
end

total_symm_vec = vcat(radsymmvec,angularsymmvec)

88-element Vector{Any}:
 RadialType2{Float64}(0.001, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.001, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.02, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.035, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.1, 11.338, [1.0, 0.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 1.0])
 RadialType2{Float64}(0.4, 11.338, [1.0, 0.0])
 ⋮
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 2.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 1.0, 0.0])
 AngularType3{Float64}(0.08, -1.0, 4.0, 11.338, [1.0, 0.0, 0.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 11.338, [1.0, 1.0, 1.0])
 AngularType3{Float64}(0.08, 1.0, 4.0, 

In [15]:
function symm_functions(positions,dis2_mat,index,symm_func_vec)
    G_vec = calc_symmetry_function.(Ref(positions),Ref(dis2_mat),Ref(index),symm_func_vec)
    return G_vec
end

symm_functions (generic function with 1 method)

In [16]:
symm_functions(positions,dis2mat,1,total_symm_vec)

88-element Vector{Float64}:
 14.640666117305633
  0.0
  7.169783183727495
  0.0
  4.524213051903176
  0.0
  0.9725333082013508
  0.0
  0.0017948851872611955
  0.0
  ⋮
  0.007425477011199379
  0.0
  0.0
  0.0009478536451217325
  0.0
  0.0
  0.02544139990466731
  0.0
  0.0

In [17]:
@benchmark symm_functions($positions,$dis2mat,rand(1:55),$total_symm_vec)

BenchmarkTools.Trial: 6823 samples with 1 evaluation.
 Range (min … max):  567.486 μs …   2.276 ms  ┊ GC (min … max): 0.00% … 58.73%
 Time  (median):     707.561 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   731.667 μs ± 141.202 μs  ┊ GC (mean ± σ):  1.14% ±  4.99%

                    ▁▅█▄▁                                        
  ▁▃▃▅▇▇▅▄▃▂▂▁▁▁▁▃▅▇█████▇▄▃▂▂▁▁▁▁▁▁▁▂▂▃▄▄▄▅▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  567 μs           Histogram: frequency by time          981 μs <

 Memory estimate: 620.02 KiB, allocs estimate: 250.

In [18]:
@everywhere function calc_thr_symm_func(positions,dis2_mat,index,symm_func::RadialSymmFunction)
    return calc_symmetry_function(positions,dis2_mat,index,symm_func)
end

In [19]:
@everywhere function thr_symm_functions(positions,dis2_mat,index,symm_func_vec)
    G_vec = zeros(length(symm_func_vec))
    for idx in eachindex(symm_func_vec)
        G_vec[idx] = calc_thr_symm_func(positions,dis2_mat,index,symm_func_vec[idx])
    end
    return G_vec
end

In [20]:
@benchmark thr_symm_functions($positions,$dis2mat,rand(1:55),$total_symm_vec)


BenchmarkTools.Trial: 4740 samples with 1 evaluation.
 Range (min … max):  708.877 μs …   6.119 ms  ┊ GC (min … max): 0.00% … 55.47%
 Time  (median):     905.287 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.053 ms ± 528.315 μs  ┊ GC (mean ± σ):  3.82% ±  8.21%

  ▄▇██▇▇▅▂     ▁▁▁▁▁▁▁▁                                         ▂
  ██████████▇█▇██████████▇▇█▇▆▃▇▆▇▆▄▅▅▃▁▅▅▆▄▄▅▅▆▁▅▅▅▁▃▅▅▁▁▄▄▅▆▆ █
  709 μs        Histogram: log(frequency) by time       4.07 ms <

 Memory estimate: 1.26 MiB, allocs estimate: 4010.